# Assistive Technology (2023-03-24)

## Email System

`Install-Package MailKit`

In [12]:
#r "nuget:MailKit"
#r "nuget:DotNetEnv"

// Load .env file
DotNetEnv.Env.TraversePath().Load();

Installed Packages DotNetEnv, 2.5.0 MailKit, 3.6.0

## Sending Mail

In [13]:
using MailKit.Net.Smtp;
using MailKit;
using MimeKit;

var virtualMorseVersion = "2023";

var message = new MimeMessage();
message.From.Add(new MailboxAddress("Sender Name", DotNetEnv.Env.GetString("EMAIL__ACCOUNT") + "@gmail.com"));
message.To.Add(new MailboxAddress("Receiver Name", DotNetEnv.Env.GetString("EMAIL__ACCOUNT") + "@gmail.com"));
message.Subject = "This message sent with Virtual Morse " + virtualMorseVersion;

message.Body = new TextPart("plain") {
    Text = @"Testing contents of the message."
};

using (var client = new SmtpClient()) {
    client.Connect("smtp.gmail.com", 587);

    client.AuthenticationMechanisms.Remove("XOAUTH2");

    // Note: only needed if the SMTP server requires authentication.
    client.Authenticate(DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    // Speaks name & email destination (spells out email address).
    Console.WriteLine(">> Sending email to [LOCATION].");

    // client.Send(message);
    client.Disconnect(true);
}

Console.WriteLine(">> Sent.");

>> Sending email to [LOCATION].
>> Sent.


## Gather Data From Specific Email
- [ ] Create, address, send
- [ ] Check messages
- [ ] Read (text-to-speech)
- [x] Delete mail
- [ ] Reply 

#### Checks Email

In [14]:
using MimeKit;
using MailKit;
using MailKit.Search;
using MailKit.Net.Imap;

using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate(DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    var inbox = client.Inbox;
    inbox.Open (FolderAccess.ReadOnly);

    Console.WriteLine(">> Checking mail server...");

    var unreadEmails = inbox.Search (SearchQuery.NotSeen);

    if (unreadEmails.Count == 1) {
        Console.WriteLine(">> You have {0} unread email.", unreadEmails.Count);
    } else {
        Console.WriteLine(">> You have {0} unread emails.", unreadEmails.Count);
    }

    var totalMessages = inbox.Count;

    // FIXME: How to deal with threaded conversations within the inbox.

    Console.WriteLine(">> Total messages: {0}", totalMessages);

    client.Disconnect(true);
}

>> Checking mail server...
>> You have 0 unread emails.
>> Total messages: 8


#### Deletes Email

In [15]:
// Delete Single Email
// https://github.com/jstedfast/MailKit/issues/552

using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate(DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    var inbox = client.Inbox;
    inbox.Open(FolderAccess.ReadWrite);

    var emailNumber = 7;

    inbox.AddFlags(emailNumber, MessageFlags.Deleted, true);

    inbox.Expunge();

    Console.WriteLine(">> Deleted.");

    client.Disconnect (true);
}

>> Deleted.


#### Reads Email Headers

In [16]:
using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate (DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    var inbox = client.Inbox;
    inbox.Open (FolderAccess.ReadOnly);

    // Email header number 
    // Oldest email = 1 vs. newest email = nth inbox location.

    var emailNumber = 1;
    var message = inbox.GetMessage(emailNumber);
    var dateSent = message.Date;
    var senderName = message.From;
    var senderAddress = message.From.Mailboxes.FirstOrDefault().Address;
    var subjectLine = message.Subject;

    Console.WriteLine("Date Sent: {0}", dateSent);
    Console.WriteLine("Sender Name: {0}", senderName);
    Console.WriteLine("Sender Address: {0}", senderAddress);
    Console.WriteLine("Subject Line: {0}", subjectLine);

    client.Disconnect (true);
}

Date Sent: 3/23/2023 7:30:20 AM +00:00
Sender Name: "Google" <no-reply@accounts.google.com>
Sender Address: no-reply@accounts.google.com
Subject Line: Recovery email verified for your Google Account


#### Reads Email

In [22]:
using (var client = new ImapClient ()) {
    client.Connect ("imap.gmail.com", 993, true);

    client.Authenticate (DotNetEnv.Env.GetString("EMAIL__ACCOUNT"), DotNetEnv.Env.GetString("APP__PASSWORD"));

    var inbox = client.Inbox;
    inbox.Open(FolderAccess.ReadOnly);

    // Email header number 
    // Oldest email = 1 vs. newest email = nth inbox location.

    // Decrement emailNumber by 1 to get the correct email header number.
    var emailNumber = 7;
    emailNumber--;

    var message = inbox.GetMessage(emailNumber);
    var body = message.TextBody;

    // TODO: Detect hyperlinks in email body and avoid reading them out loud.

    Console.WriteLine("Body: {0}", body);

    client.Disconnect(true);
}

Body: Lorem ipsum dolor sit amet.



#### Replies to Email

In [18]:
// PROGRAM STRUCTURE

// 1. Content

var content = "Sample text.";
Console.WriteLine(">> " + content);

// 2. Index Number (# of email).

var emailNumber = 0;
// Always decrement emailNumber by 1 to get the correct email header number.
emailNumber--;

Console.WriteLine(">> " + emailNumber + " is the email number entered.");

// 3. Keyed Letter

// 4. Command Switch

// 5. Program Speaks

Console.WriteLine(">> Reply to email number " + emailNumber + " from [CONTACT_NAME] [EMAIL_ADDRESS].");

// 5a. Space Pressed

Console.WriteLine(">> Reply to [CONTACT_NAME] has been sent.");

// 5b. Backspace Pressed

Console.WriteLine(">> Reply to [CONTACT_NAME] has been cancelled.");

// Email Improvements

// TODO: Read through content & distinguish original message from reply or pause.

Console.WriteLine(">> " + emailNumber + " sent.");

>> Sample text.
>> -1 is the email number entered.
>> Reply to email number -1 from [CONTACT_NAME] [EMAIL_ADDRESS].
>> Reply to [CONTACT_NAME] has been sent.
>> Reply to [CONTACT_NAME] has been cancelled.
>> -1 sent.


#### Adds Email Address to Nickname

In [36]:
// vmAddressBook.cls

using System;
using System.IO;

string contents = File.ReadAllText("AddressBook.txt");

// Pull data from AddressBook.txt, split on space into key value pairs.

Console.WriteLine(contents);

// Create dictionary to store data.

var addressBook = new Dictionary<string, string>();

// Refresh with values from text file.

var split = contents.Split(' ');

// Add values to dictionary.

addressBook.Add(split[0], split[1]);

// Nickname vs. Email (User Input)

var nickname = "John Doe";
var email = "name@example.com";

addressBook.Add(nickname, email);

// Export results to file.

Console.WriteLine(">> Nickname entered: " + nickname);

// TODO: Update value in text file.

Nickname email@example.com

>> Nickname entered: John Doe


#### Ties Email Address to Nickname

In [23]:
// Accessible file storing nicknames + addresses (updated manually).

Console.WriteLine(">> Added Email Address");

Console.WriteLine(">> [SPEAK COMPLETE ADDRESS]");

>> Added Email Address
>> [SPEAK COMPLETE ADDRESS]
